In [1]:
# ================================
# 1. 라이브러리 & 시드 설정
# ================================
import os
import json
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import copy

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


# ================================
# 2. Dataset 정의 (크롭 제거 버전)
# ================================
class EmotionDataset(Dataset):
    def __init__(self, img_root, label_root, split="train", transform=None):
        self.img_root = os.path.join(img_root, split)
        self.label_root = os.path.join(label_root, split)
        self.transform = transform
        self.emotions = ["anger", "happy", "panic", "sadness"]
        self.samples = []

        for emo in self.emotions:
            json_name = f"{split}_{emo}.json" if split != "test" else f"test_{emo}_agg.json"
            json_path = os.path.join(self.label_root, json_name)
            if not os.path.exists(json_path):
                print(f"[WARN] json not found: {json_path}")
                continue

            with open(json_path, "r", encoding="utf-8") as f:
                items = json.load(f)

            for it in items:
                filename = it["filename"]
                img_dir = os.path.join(self.img_root, emo)
                img_path = os.path.join(img_dir, filename)

                if not os.path.exists(img_path):
                    continue

                # happy=1, others=0
                label = 1 if emo == "happy" else 0

                self.samples.append((img_path, label))

        print(f"{split} samples: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label


# ================================
# 3. Transform / Dataset / DataLoader
# ================================
# ResNet 기본 입력 크기 224x224 기준
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# ★ 경로 확인! ★
img_root = "/workspace/merge_data/data/img"
label_root = "/workspace/merge_data/data/label"

train_dataset = EmotionDataset(img_root, label_root, split="train", transform=train_transform)
val_dataset   = EmotionDataset(img_root, label_root, split="val",   transform=val_transform)

batch_size = 32
num_workers = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False,
                          num_workers=num_workers)

print("\n✅ 전처리 및 데이터 로드 완료. 다음 블록으로 진행하세요.")

device: cpu
train samples: 5841
val samples: 1151

✅ 전처리 및 데이터 로드 완료. 다음 블록으로 진행하세요.


In [ ]:
# ================================
# 4. 모델 정의 (ResNet18, 2-class)
# ================================
# 사전학습된 resnet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 fc를 2클래스로 교체 (HAPPY vs OTHERS)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)

# 가중치 없는 기본 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        # ⭐️ torch.inference_mode() 적용 ⭐️
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_resnet_happy_vs_others.pth")
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

# ⭐️ torch.inference_mode() 적용 ⭐️
with torch.inference_mode():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))


Epoch 1/10
------------------------------


100%|██████████| 183/183 [10:14<00:00,  3.36s/it]

Train Loss: 0.2967 Acc: 0.8627


Val   Loss: 0.1802 Acc: 0.9314
>> best model updated

Epoch 2/10
------------------------------


100%|██████████| 183/183 [10:28<00:00,  3.43s/it]

...

Train Loss: 0.0153 Acc: 0.9955


Val   Loss: 0.2029 Acc: 0.9479

Epoch 10/10
------------------------------


100%|██████████| 183/183 [12:35<00:00,  4.13s/it]

Train Loss: 0.0191 Acc: 0.9926


Val   Loss: 0.2565 Acc: 0.9470

Best val Acc: 0.9540

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9540
Precision: 0.9073
Recall   : 0.9164
F1-score : 0.9118

Confusion Matrix (rows: true, cols: pred)
[[824  28]
 [ 25 274]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.97      0.97      0.97       852
       HAPPY       0.91      0.92      0.91       299

    accuracy                           0.95      1151
   macro avg       0.94      0.94      0.94      1151
weighted avg       0.95      0.95      0.95      1151

